In [1]:
import findspark

import pandas
import plotly as p
import plotly.plotly as py
from plotly.graph_objs import *
import plotly.figure_factory as ff
import plotly.plotly as py
import plotly.graph_objs as go
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from plotly.widgets import GraphWidget

import matplotlib.pyplot as plt  # mathplotlib

findspark.init("C:\spark-2.2.1-bin-hadoop2.7")
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.types import *
sparkSession = SparkSession.builder.appName(
    "option-pricer-write-to-hadoop").getOrCreate()

resultsHiveDF = sparkSession.read.format('json').load('E:\ProjectDB')

resultsHiveDF.createOrReplaceTempView("optionData")

<IPython.core.display.Javascript object>

In [2]:
tickers = sparkSession.sql("SELECT Ticker FROM optionData").collect()
tickerSymbols = []
for i in tickers: # remove Row(Ticker=) from ticker
   tickerSymbols.append(i['Ticker'])

expDate = sparkSession.sql("SELECT ExpirationDate FROM optionData WHERE Ticker Like 'MMM'").collect()

riskFreeRates = sparkSession.sql("SELECT riskFreeRates FROM optionData WHERE Ticker Like 'MMM'").collect()

strikePrices = sparkSession.sql("SELECT Prices['StrikePrice'] FROM optionData WHERE Ticker Like 'ZTS'").collect()

numOfDays = sparkSession.sql("SELECT NumberOfDays FROM optionData WHERE Ticker Like 'ABBV'").collect()



In [3]:
# Cell to declare all funcions

# Function to get prices based on a specific ticker symbol
def getOptionPricesByTicker(symbol):
    return sparkSession.sql("SELECT Prices FROM optionData WHERE Ticker Like '" + symbol + "'").collect()

# Function to turn int to array of ints for number of days before expDate
# eg. number of says is 5 returns [1,2,3,4,5]
def getNumberDaysArray(x):
    numDaysArr = []
    for i in range(1, x[0]['NumberOfDays'] + 2):
        numDaysArr.append(i)
    return numDaysArr

def getPricesByStrike(optionPrices, strike):
    allPrices = []
    for i in optionPrices:
        if i!= None and i['StrikePrice'] == strike:
            allPrices.append(i['Put'])
    for i in allPrices:
        if i is None:
            allPrices.remove(i)
    for i in allPrices: # for some reason have to run twice as it skips last None in above 
        if i is None:
            allPrices.remove(i)
    return allPrices



def getPricesByStrikeAndRate(optionPrices, strike, RiskFreeRate):
    allPrices = []
    for i in optionPrices[0][0]:
        if i != None and i['StrikePrice'] == strike and i['RiskFreeRate'] == RiskFreeRate:
            allPrices.append(i[0])
    return allPrices

def getCalls(optionPrices):
    callPrices = []
    for i in optionPrices[0][0]:
        if i['Call'] != None:
            callPrices.append(i['Call'])
    return callPrices

def getPuts(optionPrices):
    putPrices = []
    for i in optionPrices[0][0]:
        if i['Put'] != None:
            putPrices.append(i['Put'])
    return putPrices

def getCalls2(optionPrices):
    callPrices = []
    for i in optionPrices[0][0]:
        if i['Call'] != None:
            callPrices.append(i)
    return callPrices

def getPuts2(optionPrices):
    putPrices = []
    for i in optionPrices[0][0]:
        if i['Put'] != None:
            putPrices.append(i)
    return putPrices

def priceToArray(opPrices):
    prices = {}
    count = 1
    for i in opPrices:
        priceArr = []
        for x in i:
            if x is not None:
                priceArr.append(x)
        prices['opt'+str(count)] = (priceArr.copy())
        count = count + 1
    return prices


def riskRateToArray(rates):
    riskRatesArray = []
    for i in rates[0]['riskFreeRates']:
        riskRatesArray.append(i)
    return riskRatesArray

def getSinglePut(optionPrices):
    putPrices = []
    for i in optionPrices[0]:
        if i != None:
            putPrices.append(i)
    return putPrices

def getSingleCall(optionPrices):
    putPrices = []
    for i in optionPrices[0]:
        if i != None:
            putPrices.append(i)
    return putPrices

def getStrikePriceUnique(prices):
    strikePricesArray = []
    for i in prices[0]['Prices.StrikePrice']:
        if i != None and i not in strikePricesArray:
            strikePricesArray.append(i)
    return strikePricesArray 

def getRiskRateForGraph(ticker): #Probably have to change if more than 2 lines as graph code might crash TEST THIS
    ratesArr = []
    count = 1
    riskFreeRates = sparkSession.sql("SELECT riskFreeRates FROM optionData WHERE Ticker Like '" +
                                     ticker + "'").collect()
    for rate in riskFreeRates[0]['riskFreeRates']:
        ratesArr.append(rate)
    return ratesArr


In [4]:
# def getPricesByRiskFreeRate(optionPrices, RiskFreeRate):
#     allPrices = []
#     for i in optionPrices[0][0]:
#         if i != None and i['RiskFreeRate'] == RiskFreeRate:
#             allPrices.append(i[0])
#     return allPrices

# def priceToDic(opPrices):
#     prices = {}
#     count = 1
#     for i in opPrices:
#         priceArr = []
#         for x in i:
#             if x != "None":
#                 priceArr.append(x)
#         prices['opt'+str(count)] = (priceArr.copy())
#         count = count + 1
#     return prices


In [5]:
prices = getOptionPricesByTicker('ABBV')

callPrices = []
putPrices = []

#callPricesStrike = getPricesByStrike(prices, 65.0)



#callPricesSR = getPricesByStrikeAndRate(prices, 67.5, 1.67)

callPrices = getCalls(prices)
callPricesRate = priceToDic(getPricesByRiskFreeRate(prices, 1.67))
#print(callPricesRate )

putPrices = getPuts(prices)
        
numberOfDays = getNumberDaysArray(numOfDays)
    

NameError: name 'priceToDic' is not defined

In [6]:
# def getOptionPricesByTicker(symbol):
#     return sparkSession.sql("SELECT Prices FROM optionData WHERE Ticker Like '" + symbol + "'").collect()

# print(getOptionPricesByTicker('AMZN'))

In [7]:
# Function to get prices based on a specific ticker symbol
def getOptionPricesByTicker(symbol):
    return sparkSession.sql("SELECT Prices FROM optionData WHERE Ticker Like '" + symbol + "'").collect()

def getCalls(optionPrices):
    callPrices = []
    for i in optionPrices[0][0]:
        if i['Call'] != None:
            callPrices.append(i['Call'])
    return callPrices

def getPuts(optionPrices):
    putPrices = []
    for i in optionPrices[0][0]:
        if i['Put'] != None:
            putPrices.append(i['Put'])
    return putPrices

def getCalls2(optionPrices):
    callPrices = []
    for i in optionPrices[0][0]:
        if i['Call'] != None:
            callPrices.append(i)
    return callPrices

def getPuts2(optionPrices):
    putPrices = []
    for i in optionPrices[0][0]:
        if i['Put'] != None:
            putPrices.append(i)
    return putPrices

def priceToArray(opPrices):
    prices = {}
    count = 1
    for i in opPrices:
        priceArr = []
        for x in i:
            if x is not None:
                priceArr.append(x)
        prices['opt'+str(count)] = (priceArr.copy())
        count = count + 1
    return prices

In [8]:
# too much options to remove
def graphCallsByTicker(tickerSymbol):
    opPriceDic = getOptionPricesByTicker(tickerSymbol)
    pp = getCalls(opPriceDic)
    ppDic = priceToArray(pp)
    numOfDays = sparkSession.sql("SELECT NumberOfDays FROM optionData WHERE Ticker Like '" + tickerSymbol + "'").collect()
    numberOfDays = getNumberDaysArray(numOfDays)
    for key, values in ppDic.items():
        plt.plot(numberOfDays, values)
    plt.xlabel('Days to Expiration')
    plt.ylabel('Option Prices')
    plt.title('Calls Options for '+ tickerSymbol)
    #plt.ticklabel_format(style='plain')
    plt.show()

In [9]:
interact(graphCallsByTicker, tickerSymbol=tickerSymbols);

interactive(children=(Dropdown(description='tickerSymbol', options=('AMZN', 'BRK.B', 'GOOG', 'BA', 'MLM', 'XL', 'HSIC', 'MTD', 'CMG', 'IDXX', 'PDCO', 'FLIR', 'CMA', 'INTU', 'KHC', 'FL', 'ZION', 'ROK', 'MAS', 'EL', 'ARNC', 'FOX', 'ADBE', 'HLT', 'RE', 'MCHP', 'JCI', 'BHGE', 'LLL', 'SPG', 'DVA', 'TJX', 'ALK', 'XRX', 'ALL', 'TRV', 'PPL', 'DRI', 'MSI', 'SEE', 'FMC', 'ANSS', 'HRB', 'LNC', 'INCY', 'ABBV', 'MTB', 'EQIX', 'WM', 'AVB', 'RTN', 'EIX', 'ALB', 'GGP', 'HBI', 'FTI', 'IVZ', 'DUK', 'UHS', 'PCLN', 'C', 'NTRS', 'CCI', 'AON', 'COL', 'FE', 'EXPE', 'PH', 'QRVO', 'AYI', 'CPB', 'SO', 'AVY', 'SJM', 'DG', 'NWL', 'D', 'LEN', 'SIG', 'DLR', 'GIS', 'WLTW', 'ESS', 'OMC', 'TEL', 'VRSN', 'AAP', 'JWN', 'CELG', 'BWA', 'ARE', 'IPG', 'FISV', 'ORLY', 'MU', 'JEC', 'EQR', 'CNC', 'CTAS', 'SBAC', 'APH', 'NFLX', 'LB', 'STX', 'RMD', 'WRK', 'BXP', 'PSA', 'SYK', 'SRE', 'PVH', 'PKG', 'KLAC', 'TMO', 'EMN', 'VMC', 'GOOGL', 'NOC', 'HBAN', 'STI', 'AMP', 'SNA', 'PNC', 'CB', 'AZO', 'TDG', 'ROST', 'CHRW', 'DHR', 'XYL', 'MCO', 'IR', 'WDC', 'BCR', 'CXO', 'NFX', 'K', 'FITB', 'PRU', 'A', 'BEN', 'NVDA', 'DWDP', 'APD', 'WYN', 'AGN', 'NI', 'HRS', 'AOS', 'IT', 'ADS', 'SWKS', 'CTXS', 'NDAQ', 'HP', 'LH', 'HST', 'PAYX', 'EXC', 'ROP', 'UTX', 'VTR', 'IRM', 'PWR', 'CA', 'RSG', 'WMT', 'COO', 'JBHT', 'O', 'ADSK', 'XEC', 'ICE', 'RF', 'NEE', 'FB', 'PFG', 'XOM', 'DISCA', 'STT', 'CDNS', 'CBG', 'CFG', 'HCP', 'MSFT', 'ZBH', 'MHK', 'ETR', 'EQT', 'LOW', 'CSX', 'FIS', 'RJF', 'AMG', 'XRAY', 'LRCX', 'IFF', 'HOLX', 'PCAR', 'GPN', 'CVS', 'EMR', 'SCG', 'FLS', 'CERN', 'CRM', 'CAG', 'PYPL', 'ISRG', 'FAST', 'NCLH', 'DTE', 'KIM', 'DXC', 'MKC', 'NLSN', 'AAPL', 'REG', 'BIIB', 'MMC', 'INTC', 'PNW', 'QCOM', 'PGR', 'HCN', 'AMAT', 'CI', 'RHT', 'PSX', 'CHD', 'ATVI', 'AVGO', 'REGN', 'GS', 'NKE', 'NWSA', 'WEC', 'WFC', 'UAL', 'TGT', 'PNR', 'DLTR', 'NBL', 'BLK', 'BBT', 'KSU', 'UNP', 'ALLE', 'WYNN', 'CAT', 'AEP', 'JPM', 'SNPS', 'JNJ', 'AWK', 'ALGN', 'EVHC', 'ETN', 'WAT', 'NTAP', 'HAS', 'SRCL', 'LMT', 'PG', 'FRT', 'APA', 'ORCL', 'DIS', 'DGX', 'EOG', 'AFL', 'EA', 'NUE', 'AAL', 'WBA', 'CTSH', 'VLO', 'HD', 'SBUX', 'CVX', 'COF', 'TSS', 'OXY', 'V', 'AXP', 'BDX', 'DISCK', 'MMM', 'NAVI', 'HAL', 'LUV', 'BHF', 'AKAM', 'ANDV', 'ABC', 'BBY', 'AJG', 'INFO', 'CSCO', 'TXN', 'LNT', 'AIZ', 'MCD', 'APC', 'MS', 'M', 'MDT', 'ES', 'ESRX', 'DE', 'BAC', 'ILMN', 'DLPH', 'PEP', 'SHW', 'ADI', 'GM', 'RCL', 'KSS', 'CSRA', 'DISH', 'DHI', 'ITW', 'IBM', 'DPS', 'GWW', 'MNST', 'SCHW', 'MAR', 'KORS', 'EXR', 'KR', 'TIF', 'BLL', 'CF', 'MA', 'VAR', 'PM', 'MAA', 'MAC', 'AME', 'PKI', 'DAL', 'AMGN', 'LEG', 'CHTR', 'UNH', 'HES', 'URI', 'TXT', 'CCL', 'COP', 'MPC', 'ETFC', 'MRK', 'VRTX', 'COST', 'TSCO', 'FFIV', 'CBOE', 'PBCT', 'WU', 'BMY', 'AMT', 'GILD', 'NSC', 'LKQ', 'AMD', 'LLY', 'CAH', 'ALXN', 'YUM', 'CBS', 'HOG', 'VZ', 'CMCSA', 'SLG', 'MCK', 'VRSK', 'VFC', 'LUK', 'PLD', 'FBHS', 'RHI', 'SNI', 'AES', 'CMI', 'GE', 'CNP', 'EFX', 'UA', 'T', 'XLNX', 'TRIP', 'FTV', 'HON', 'MGM', 'KMX', 'IP', 'HCA', 'EW', 'DVN', 'MO', 'DFS', 'ADP', 'TAP', 'AIG', 'GRMN', 'KO', 'TROW', 'CL', 'KMB', 'CLX', 'UDR', 'ED', 'ZTS', 'PCG', 'ACN', 'EBAY', 'OKE', 'CME', 'VNO', 'NEM', 'WHR', 'PFE', 'HSY', 'UAA', 'MET', 'FDX', 'PEG', 'MON', 'XEL', 'PPG', 'SLB', 'UPS', 'TPR', 'GPS', 'SYF', 'FCX', 'TSN', 'ABT', 'DOV', 'MYL', 'RL', 'LYB', 'BK', 'PXD', 'GLW', 'ANTM', 'SYY', 'GPC', 'MDLZ', 'CINF', 'GD', 'EXPD', 'NOV', 'BAX', 'AEE', 'L', 'ULTA', 'COTY', 'ECL', 'USB', 'VIAB', 'UNM', 'PX', 'MRO', 'SPGI', 'MAT', 'PRGO', 'WMB', 'PHM', 'GT', 'HIG', 'DRE', 'CTL', 'ADM', 'TMK', 'KMI', 'CMS', 'AIV', 'HPE', 'HPQ', 'BSX', 'FLR', 'TWX', 'AET', 'SYMC', 'HRL', 'STZ', 'JNPR', 'WY', 'CHK', 'RRC', 'KEY', 'COG', 'NRG', 'NWS', 'FOXA', 'F', 'SWK', 'HUM', 'AVGO', 'DGX', 'MOS'), value='AMZN'), Output()), _dom_classes=('widget-interact',))

In [17]:
# too much options to remove
import numpy as np
def graphPutsByTicker(tickerSymbol):
    opPriceDic = priceToArray(getPuts(getOptionPricesByTicker(tickerSymbol)))
    numOfDays = sparkSession.sql("SELECT NumberOfDays FROM optionData WHERE Ticker Like '" + tickerSymbol + "'").collect()
    numberOfDays = getNumberDaysArray(numOfDays)
    for key, values in opPriceDic.items():
        plt.plot(numberOfDays, values)
    plt.xlabel('Days to Expiration')
    plt.ylabel('Option Prices')
    plt.title('Put Options for '+ tickerSymbol)
#     t = np.arange(0.01, 20.0, 0.01)
#     plt.semilogy(t, np.exp(-t/5.0))
    plt.show()

In [18]:
interact(graphPutsByTicker, tickerSymbol=tickerSymbols);

interactive(children=(Dropdown(description='tickerSymbol', options=('AMZN', 'BRK.B', 'GOOG', 'BA', 'MLM', 'XL', 'HSIC', 'MTD', 'CMG', 'IDXX', 'PDCO', 'FLIR', 'CMA', 'INTU', 'KHC', 'FL', 'ZION', 'ROK', 'MAS', 'EL', 'ARNC', 'FOX', 'ADBE', 'HLT', 'RE', 'MCHP', 'JCI', 'BHGE', 'LLL', 'SPG', 'DVA', 'TJX', 'ALK', 'XRX', 'ALL', 'TRV', 'PPL', 'DRI', 'MSI', 'SEE', 'FMC', 'ANSS', 'HRB', 'LNC', 'INCY', 'ABBV', 'MTB', 'EQIX', 'WM', 'AVB', 'RTN', 'EIX', 'ALB', 'GGP', 'HBI', 'FTI', 'IVZ', 'DUK', 'UHS', 'PCLN', 'C', 'NTRS', 'CCI', 'AON', 'COL', 'FE', 'EXPE', 'PH', 'QRVO', 'AYI', 'CPB', 'SO', 'AVY', 'SJM', 'DG', 'NWL', 'D', 'LEN', 'SIG', 'DLR', 'GIS', 'WLTW', 'ESS', 'OMC', 'TEL', 'VRSN', 'AAP', 'JWN', 'CELG', 'BWA', 'ARE', 'IPG', 'FISV', 'ORLY', 'MU', 'JEC', 'EQR', 'CNC', 'CTAS', 'SBAC', 'APH', 'NFLX', 'LB', 'STX', 'RMD', 'WRK', 'BXP', 'PSA', 'SYK', 'SRE', 'PVH', 'PKG', 'KLAC', 'TMO', 'EMN', 'VMC', 'GOOGL', 'NOC', 'HBAN', 'STI', 'AMP', 'SNA', 'PNC', 'CB', 'AZO', 'TDG', 'ROST', 'CHRW', 'DHR', 'XYL', 'MCO', 'IR', 'WDC', 'BCR', 'CXO', 'NFX', 'K', 'FITB', 'PRU', 'A', 'BEN', 'NVDA', 'DWDP', 'APD', 'WYN', 'AGN', 'NI', 'HRS', 'AOS', 'IT', 'ADS', 'SWKS', 'CTXS', 'NDAQ', 'HP', 'LH', 'HST', 'PAYX', 'EXC', 'ROP', 'UTX', 'VTR', 'IRM', 'PWR', 'CA', 'RSG', 'WMT', 'COO', 'JBHT', 'O', 'ADSK', 'XEC', 'ICE', 'RF', 'NEE', 'FB', 'PFG', 'XOM', 'DISCA', 'STT', 'CDNS', 'CBG', 'CFG', 'HCP', 'MSFT', 'ZBH', 'MHK', 'ETR', 'EQT', 'LOW', 'CSX', 'FIS', 'RJF', 'AMG', 'XRAY', 'LRCX', 'IFF', 'HOLX', 'PCAR', 'GPN', 'CVS', 'EMR', 'SCG', 'FLS', 'CERN', 'CRM', 'CAG', 'PYPL', 'ISRG', 'FAST', 'NCLH', 'DTE', 'KIM', 'DXC', 'MKC', 'NLSN', 'AAPL', 'REG', 'BIIB', 'MMC', 'INTC', 'PNW', 'QCOM', 'PGR', 'HCN', 'AMAT', 'CI', 'RHT', 'PSX', 'CHD', 'ATVI', 'AVGO', 'REGN', 'GS', 'NKE', 'NWSA', 'WEC', 'WFC', 'UAL', 'TGT', 'PNR', 'DLTR', 'NBL', 'BLK', 'BBT', 'KSU', 'UNP', 'ALLE', 'WYNN', 'CAT', 'AEP', 'JPM', 'SNPS', 'JNJ', 'AWK', 'ALGN', 'EVHC', 'ETN', 'WAT', 'NTAP', 'HAS', 'SRCL', 'LMT', 'PG', 'FRT', 'APA', 'ORCL', 'DIS', 'DGX', 'EOG', 'AFL', 'EA', 'NUE', 'AAL', 'WBA', 'CTSH', 'VLO', 'HD', 'SBUX', 'CVX', 'COF', 'TSS', 'OXY', 'V', 'AXP', 'BDX', 'DISCK', 'MMM', 'NAVI', 'HAL', 'LUV', 'BHF', 'AKAM', 'ANDV', 'ABC', 'BBY', 'AJG', 'INFO', 'CSCO', 'TXN', 'LNT', 'AIZ', 'MCD', 'APC', 'MS', 'M', 'MDT', 'ES', 'ESRX', 'DE', 'BAC', 'ILMN', 'DLPH', 'PEP', 'SHW', 'ADI', 'GM', 'RCL', 'KSS', 'CSRA', 'DISH', 'DHI', 'ITW', 'IBM', 'DPS', 'GWW', 'MNST', 'SCHW', 'MAR', 'KORS', 'EXR', 'KR', 'TIF', 'BLL', 'CF', 'MA', 'VAR', 'PM', 'MAA', 'MAC', 'AME', 'PKI', 'DAL', 'AMGN', 'LEG', 'CHTR', 'UNH', 'HES', 'URI', 'TXT', 'CCL', 'COP', 'MPC', 'ETFC', 'MRK', 'VRTX', 'COST', 'TSCO', 'FFIV', 'CBOE', 'PBCT', 'WU', 'BMY', 'AMT', 'GILD', 'NSC', 'LKQ', 'AMD', 'LLY', 'CAH', 'ALXN', 'YUM', 'CBS', 'HOG', 'VZ', 'CMCSA', 'SLG', 'MCK', 'VRSK', 'VFC', 'LUK', 'PLD', 'FBHS', 'RHI', 'SNI', 'AES', 'CMI', 'GE', 'CNP', 'EFX', 'UA', 'T', 'XLNX', 'TRIP', 'FTV', 'HON', 'MGM', 'KMX', 'IP', 'HCA', 'EW', 'DVN', 'MO', 'DFS', 'ADP', 'TAP', 'AIG', 'GRMN', 'KO', 'TROW', 'CL', 'KMB', 'CLX', 'UDR', 'ED', 'ZTS', 'PCG', 'ACN', 'EBAY', 'OKE', 'CME', 'VNO', 'NEM', 'WHR', 'PFE', 'HSY', 'UAA', 'MET', 'FDX', 'PEG', 'MON', 'XEL', 'PPG', 'SLB', 'UPS', 'TPR', 'GPS', 'SYF', 'FCX', 'TSN', 'ABT', 'DOV', 'MYL', 'RL', 'LYB', 'BK', 'PXD', 'GLW', 'ANTM', 'SYY', 'GPC', 'MDLZ', 'CINF', 'GD', 'EXPD', 'NOV', 'BAX', 'AEE', 'L', 'ULTA', 'COTY', 'ECL', 'USB', 'VIAB', 'UNM', 'PX', 'MRO', 'SPGI', 'MAT', 'PRGO', 'WMB', 'PHM', 'GT', 'HIG', 'DRE', 'CTL', 'ADM', 'TMK', 'KMI', 'CMS', 'AIV', 'HPE', 'HPQ', 'BSX', 'FLR', 'TWX', 'AET', 'SYMC', 'HRL', 'STZ', 'JNPR', 'WY', 'CHK', 'RRC', 'KEY', 'COG', 'NRG', 'NWS', 'FOXA', 'F', 'SWK', 'HUM', 'AVGO', 'DGX', 'MOS'), value='AMZN'), Output()), _dom_classes=('widget-interact',))

In [15]:
# def getPricesByStrike2(optionPrices, strike): # Think i can remove
#     allPrices = []
#     for i in optionPrices:
#         if i!= None and i['StrikePrice'] == strike:
#             allPrices.append(i['Put'])
#     for i in allPrices:
#         if i is None:
#             allPrices.remove(i)
#     for i in allPrices: # for some reason have to run twice as it skips last None in above 
#         if i is None:
#             allPrices.remove(i)
#     return allPrices


In [23]:
def priceToDicForStrikeGraph(opPrices):
    prices = {}
    count = 1
    for i in opPrices:
        priceArr = []
        for x in i:
            if x is not None:
                priceArr.append(x)
        prices['opt'+str(count)] = (priceArr.copy())
        count = count + 1
    return prices

In [24]:
def graphPutsByTickerWithStrike(strikePrice, tickerSymbol):
    prices = getOptionPricesByTicker(tickerSymbol)
    putPrices = getPuts2(prices)
    rates = getRiskRateForGraph(tickerSymbol)
    pp = getPricesByStrike( putPrices,strikePrice)
    opPriceDic = priceToDicForStrikeGraph(pp)
    numOfDays = sparkSession.sql("SELECT NumberOfDays FROM optionData WHERE Ticker Like '" +
                                 tickerSymbol + "'").collect()
    numberOfDays = getNumberDaysArray(numOfDays)
    count = 0 # to get correct rates for graph pointer texts
    for key, values in opPriceDic.items():
        textXPoint = numberOfDays[len(numberOfDays)-1] + 0.4 # cant do this in .annotate(xytext) 
        plt.plot(numberOfDays, values)
        plt.annotate('Risk rate: '+str(rates[count]), xy=(numberOfDays[len(numberOfDays)-1], values[len(values)-1]), 
             xytext=(textXPoint, values[len(values)-1]),
        arrowprops=dict(arrowstyle='wedge'), bbox=dict(boxstyle="round", fc="w"),
        )
        count = count +1
    plt.xlabel('Days to Expiration')
    plt.ylabel('Option Prices')
    plt.title('Put Options for '+ tickerSymbol + ' with Strike Price €' + str(strikePrice))
    plt.show()

In [25]:
def passTickerforPutStrike(tickerSymbol):
    strikePrices = sparkSession.sql("SELECT Prices['StrikePrice'] FROM optionData WHERE Ticker Like '"+ 
                                    tickerSymbol+"'").collect() # all use the same rates
    strikePrices = getStrikePriceUnique(strikePrices)
    interact(graphPutsByTickerWithStrike,strikePrice = strikePrices, tickerSymbol = fixed(tickerSymbol)); # Fixed removes extra interaction box

In [26]:

interact(passTickerforPutStrike,tickerSymbol=tickerSymbols);

interactive(children=(Dropdown(description='tickerSymbol', options=('AMZN', 'BRK.B', 'GOOG', 'BA', 'MLM', 'XL', 'HSIC', 'MTD', 'CMG', 'IDXX', 'PDCO', 'FLIR', 'CMA', 'INTU', 'KHC', 'FL', 'ZION', 'ROK', 'MAS', 'EL', 'ARNC', 'FOX', 'ADBE', 'HLT', 'RE', 'MCHP', 'JCI', 'BHGE', 'LLL', 'SPG', 'DVA', 'TJX', 'ALK', 'XRX', 'ALL', 'TRV', 'PPL', 'DRI', 'MSI', 'SEE', 'FMC', 'ANSS', 'HRB', 'LNC', 'INCY', 'ABBV', 'MTB', 'EQIX', 'WM', 'AVB', 'RTN', 'EIX', 'ALB', 'GGP', 'HBI', 'FTI', 'IVZ', 'DUK', 'UHS', 'PCLN', 'C', 'NTRS', 'CCI', 'AON', 'COL', 'FE', 'EXPE', 'PH', 'QRVO', 'AYI', 'CPB', 'SO', 'AVY', 'SJM', 'DG', 'NWL', 'D', 'LEN', 'SIG', 'DLR', 'GIS', 'WLTW', 'ESS', 'OMC', 'TEL', 'VRSN', 'AAP', 'JWN', 'CELG', 'BWA', 'ARE', 'IPG', 'FISV', 'ORLY', 'MU', 'JEC', 'EQR', 'CNC', 'CTAS', 'SBAC', 'APH', 'NFLX', 'LB', 'STX', 'RMD', 'WRK', 'BXP', 'PSA', 'SYK', 'SRE', 'PVH', 'PKG', 'KLAC', 'TMO', 'EMN', 'VMC', 'GOOGL', 'NOC', 'HBAN', 'STI', 'AMP', 'SNA', 'PNC', 'CB', 'AZO', 'TDG', 'ROST', 'CHRW', 'DHR', 'XYL', 'MCO', 'IR', 'WDC', 'BCR', 'CXO', 'NFX', 'K', 'FITB', 'PRU', 'A', 'BEN', 'NVDA', 'DWDP', 'APD', 'WYN', 'AGN', 'NI', 'HRS', 'AOS', 'IT', 'ADS', 'SWKS', 'CTXS', 'NDAQ', 'HP', 'LH', 'HST', 'PAYX', 'EXC', 'ROP', 'UTX', 'VTR', 'IRM', 'PWR', 'CA', 'RSG', 'WMT', 'COO', 'JBHT', 'O', 'ADSK', 'XEC', 'ICE', 'RF', 'NEE', 'FB', 'PFG', 'XOM', 'DISCA', 'STT', 'CDNS', 'CBG', 'CFG', 'HCP', 'MSFT', 'ZBH', 'MHK', 'ETR', 'EQT', 'LOW', 'CSX', 'FIS', 'RJF', 'AMG', 'XRAY', 'LRCX', 'IFF', 'HOLX', 'PCAR', 'GPN', 'CVS', 'EMR', 'SCG', 'FLS', 'CERN', 'CRM', 'CAG', 'PYPL', 'ISRG', 'FAST', 'NCLH', 'DTE', 'KIM', 'DXC', 'MKC', 'NLSN', 'AAPL', 'REG', 'BIIB', 'MMC', 'INTC', 'PNW', 'QCOM', 'PGR', 'HCN', 'AMAT', 'CI', 'RHT', 'PSX', 'CHD', 'ATVI', 'AVGO', 'REGN', 'GS', 'NKE', 'NWSA', 'WEC', 'WFC', 'UAL', 'TGT', 'PNR', 'DLTR', 'NBL', 'BLK', 'BBT', 'KSU', 'UNP', 'ALLE', 'WYNN', 'CAT', 'AEP', 'JPM', 'SNPS', 'JNJ', 'AWK', 'ALGN', 'EVHC', 'ETN', 'WAT', 'NTAP', 'HAS', 'SRCL', 'LMT', 'PG', 'FRT', 'APA', 'ORCL', 'DIS', 'DGX', 'EOG', 'AFL', 'EA', 'NUE', 'AAL', 'WBA', 'CTSH', 'VLO', 'HD', 'SBUX', 'CVX', 'COF', 'TSS', 'OXY', 'V', 'AXP', 'BDX', 'DISCK', 'MMM', 'NAVI', 'HAL', 'LUV', 'BHF', 'AKAM', 'ANDV', 'ABC', 'BBY', 'AJG', 'INFO', 'CSCO', 'TXN', 'LNT', 'AIZ', 'MCD', 'APC', 'MS', 'M', 'MDT', 'ES', 'ESRX', 'DE', 'BAC', 'ILMN', 'DLPH', 'PEP', 'SHW', 'ADI', 'GM', 'RCL', 'KSS', 'CSRA', 'DISH', 'DHI', 'ITW', 'IBM', 'DPS', 'GWW', 'MNST', 'SCHW', 'MAR', 'KORS', 'EXR', 'KR', 'TIF', 'BLL', 'CF', 'MA', 'VAR', 'PM', 'MAA', 'MAC', 'AME', 'PKI', 'DAL', 'AMGN', 'LEG', 'CHTR', 'UNH', 'HES', 'URI', 'TXT', 'CCL', 'COP', 'MPC', 'ETFC', 'MRK', 'VRTX', 'COST', 'TSCO', 'FFIV', 'CBOE', 'PBCT', 'WU', 'BMY', 'AMT', 'GILD', 'NSC', 'LKQ', 'AMD', 'LLY', 'CAH', 'ALXN', 'YUM', 'CBS', 'HOG', 'VZ', 'CMCSA', 'SLG', 'MCK', 'VRSK', 'VFC', 'LUK', 'PLD', 'FBHS', 'RHI', 'SNI', 'AES', 'CMI', 'GE', 'CNP', 'EFX', 'UA', 'T', 'XLNX', 'TRIP', 'FTV', 'HON', 'MGM', 'KMX', 'IP', 'HCA', 'EW', 'DVN', 'MO', 'DFS', 'ADP', 'TAP', 'AIG', 'GRMN', 'KO', 'TROW', 'CL', 'KMB', 'CLX', 'UDR', 'ED', 'ZTS', 'PCG', 'ACN', 'EBAY', 'OKE', 'CME', 'VNO', 'NEM', 'WHR', 'PFE', 'HSY', 'UAA', 'MET', 'FDX', 'PEG', 'MON', 'XEL', 'PPG', 'SLB', 'UPS', 'TPR', 'GPS', 'SYF', 'FCX', 'TSN', 'ABT', 'DOV', 'MYL', 'RL', 'LYB', 'BK', 'PXD', 'GLW', 'ANTM', 'SYY', 'GPC', 'MDLZ', 'CINF', 'GD', 'EXPD', 'NOV', 'BAX', 'AEE', 'L', 'ULTA', 'COTY', 'ECL', 'USB', 'VIAB', 'UNM', 'PX', 'MRO', 'SPGI', 'MAT', 'PRGO', 'WMB', 'PHM', 'GT', 'HIG', 'DRE', 'CTL', 'ADM', 'TMK', 'KMI', 'CMS', 'AIV', 'HPE', 'HPQ', 'BSX', 'FLR', 'TWX', 'AET', 'SYMC', 'HRL', 'STZ', 'JNPR', 'WY', 'CHK', 'RRC', 'KEY', 'COG', 'NRG', 'NWS', 'FOXA', 'F', 'SWK', 'HUM', 'AVGO', 'DGX', 'MOS'), value='AMZN'), Output()), _dom_classes=('widget-interact',))

In [31]:
def getPricesByStrikeCallCallByStrike(optionPrices, strike):
    allPrices = []
    for i in optionPrices:
        print(i)
        if i!= None and i['StrikePrice'] == strike:
            allPrices.append(i['Call'])
    for i in allPrices:
        if i is None:
            allPrices.remove(i)
    for i in allPrices: # for some reason have to run twice as it skips last None in above 
        if i is None:
            allPrices.remove(i)
    return allPrices


def graphCallsByTickerWithStrike(strikePrice, tickerSymbol):
    prices = getOptionPricesByTicker(tickerSymbol)
    callPrices = getCalls(prices)
    rates = getRiskRateForGraph(tickerSymbol)
    pp = getPricesByStrikeCallCallByStrike( callPrices,strikePrice)
    opPriceDic = priceToArray2(pp)
    numOfDays = sparkSession.sql("SELECT NumberOfDays FROM optionData WHERE Ticker Like '" + tickerSymbol + "'").collect()
    numberOfDays = getNumberDaysArray(numOfDays)
    count = 0 # to get correct rates for graph pointer texts
    for key, values in opPriceDic.items():
        textXPoint = numberOfDays[len(numberOfDays)-1] + 1  # cant do this in .annotate(xytext) 
        plt.plot(numberOfDays, values)
        plt.annotate('Risk rate: '+str(rates[count]), xy=(numberOfDays[len(numberOfDays)-1], values[len(values)-1]), 
                     xytext=(textXPoint, values[len(values)-1]),
            arrowprops=dict(arrowstyle='wedge'), bbox=dict(boxstyle="round", fc="w"),
            )
        count = count +1
    plt.xlabel('Days to Expiration')
    plt.ylabel('Option Prices')
    plt.title('Call Options for '+ tickerSymbol + ' with Strike Price €' + str(strikePrice))
    plt.ticklabel_format(useOffset=False)
    plt.show()
    
def passTickerforCallStrike(tickerSymbol):
    strikePrices = sparkSession.sql("SELECT Prices['StrikePrice'] FROM optionData WHERE Ticker Like '"+ tickerSymbol+"'").collect() # all use the same rates
    strikePrices = getStrikePriceUnique(strikePrices)
    interact(graphCallsByTickerWithStrike, strikePrice = strikePrices, tickerSymbol = fixed(tickerSymbol)); # Fixed removes extra interaction box);

In [32]:
interact(passTickerforCallStrike, tickerSymbol=tickerSymbols);

interactive(children=(Dropdown(description='tickerSymbol', options=('AMZN', 'BRK.B', 'GOOG', 'BA', 'MLM', 'XL', 'HSIC', 'MTD', 'CMG', 'IDXX', 'PDCO', 'FLIR', 'CMA', 'INTU', 'KHC', 'FL', 'ZION', 'ROK', 'MAS', 'EL', 'ARNC', 'FOX', 'ADBE', 'HLT', 'RE', 'MCHP', 'JCI', 'BHGE', 'LLL', 'SPG', 'DVA', 'TJX', 'ALK', 'XRX', 'ALL', 'TRV', 'PPL', 'DRI', 'MSI', 'SEE', 'FMC', 'ANSS', 'HRB', 'LNC', 'INCY', 'ABBV', 'MTB', 'EQIX', 'WM', 'AVB', 'RTN', 'EIX', 'ALB', 'GGP', 'HBI', 'FTI', 'IVZ', 'DUK', 'UHS', 'PCLN', 'C', 'NTRS', 'CCI', 'AON', 'COL', 'FE', 'EXPE', 'PH', 'QRVO', 'AYI', 'CPB', 'SO', 'AVY', 'SJM', 'DG', 'NWL', 'D', 'LEN', 'SIG', 'DLR', 'GIS', 'WLTW', 'ESS', 'OMC', 'TEL', 'VRSN', 'AAP', 'JWN', 'CELG', 'BWA', 'ARE', 'IPG', 'FISV', 'ORLY', 'MU', 'JEC', 'EQR', 'CNC', 'CTAS', 'SBAC', 'APH', 'NFLX', 'LB', 'STX', 'RMD', 'WRK', 'BXP', 'PSA', 'SYK', 'SRE', 'PVH', 'PKG', 'KLAC', 'TMO', 'EMN', 'VMC', 'GOOGL', 'NOC', 'HBAN', 'STI', 'AMP', 'SNA', 'PNC', 'CB', 'AZO', 'TDG', 'ROST', 'CHRW', 'DHR', 'XYL', 'MCO', 'IR', 'WDC', 'BCR', 'CXO', 'NFX', 'K', 'FITB', 'PRU', 'A', 'BEN', 'NVDA', 'DWDP', 'APD', 'WYN', 'AGN', 'NI', 'HRS', 'AOS', 'IT', 'ADS', 'SWKS', 'CTXS', 'NDAQ', 'HP', 'LH', 'HST', 'PAYX', 'EXC', 'ROP', 'UTX', 'VTR', 'IRM', 'PWR', 'CA', 'RSG', 'WMT', 'COO', 'JBHT', 'O', 'ADSK', 'XEC', 'ICE', 'RF', 'NEE', 'FB', 'PFG', 'XOM', 'DISCA', 'STT', 'CDNS', 'CBG', 'CFG', 'HCP', 'MSFT', 'ZBH', 'MHK', 'ETR', 'EQT', 'LOW', 'CSX', 'FIS', 'RJF', 'AMG', 'XRAY', 'LRCX', 'IFF', 'HOLX', 'PCAR', 'GPN', 'CVS', 'EMR', 'SCG', 'FLS', 'CERN', 'CRM', 'CAG', 'PYPL', 'ISRG', 'FAST', 'NCLH', 'DTE', 'KIM', 'DXC', 'MKC', 'NLSN', 'AAPL', 'REG', 'BIIB', 'MMC', 'INTC', 'PNW', 'QCOM', 'PGR', 'HCN', 'AMAT', 'CI', 'RHT', 'PSX', 'CHD', 'ATVI', 'AVGO', 'REGN', 'GS', 'NKE', 'NWSA', 'WEC', 'WFC', 'UAL', 'TGT', 'PNR', 'DLTR', 'NBL', 'BLK', 'BBT', 'KSU', 'UNP', 'ALLE', 'WYNN', 'CAT', 'AEP', 'JPM', 'SNPS', 'JNJ', 'AWK', 'ALGN', 'EVHC', 'ETN', 'WAT', 'NTAP', 'HAS', 'SRCL', 'LMT', 'PG', 'FRT', 'APA', 'ORCL', 'DIS', 'DGX', 'EOG', 'AFL', 'EA', 'NUE', 'AAL', 'WBA', 'CTSH', 'VLO', 'HD', 'SBUX', 'CVX', 'COF', 'TSS', 'OXY', 'V', 'AXP', 'BDX', 'DISCK', 'MMM', 'NAVI', 'HAL', 'LUV', 'BHF', 'AKAM', 'ANDV', 'ABC', 'BBY', 'AJG', 'INFO', 'CSCO', 'TXN', 'LNT', 'AIZ', 'MCD', 'APC', 'MS', 'M', 'MDT', 'ES', 'ESRX', 'DE', 'BAC', 'ILMN', 'DLPH', 'PEP', 'SHW', 'ADI', 'GM', 'RCL', 'KSS', 'CSRA', 'DISH', 'DHI', 'ITW', 'IBM', 'DPS', 'GWW', 'MNST', 'SCHW', 'MAR', 'KORS', 'EXR', 'KR', 'TIF', 'BLL', 'CF', 'MA', 'VAR', 'PM', 'MAA', 'MAC', 'AME', 'PKI', 'DAL', 'AMGN', 'LEG', 'CHTR', 'UNH', 'HES', 'URI', 'TXT', 'CCL', 'COP', 'MPC', 'ETFC', 'MRK', 'VRTX', 'COST', 'TSCO', 'FFIV', 'CBOE', 'PBCT', 'WU', 'BMY', 'AMT', 'GILD', 'NSC', 'LKQ', 'AMD', 'LLY', 'CAH', 'ALXN', 'YUM', 'CBS', 'HOG', 'VZ', 'CMCSA', 'SLG', 'MCK', 'VRSK', 'VFC', 'LUK', 'PLD', 'FBHS', 'RHI', 'SNI', 'AES', 'CMI', 'GE', 'CNP', 'EFX', 'UA', 'T', 'XLNX', 'TRIP', 'FTV', 'HON', 'MGM', 'KMX', 'IP', 'HCA', 'EW', 'DVN', 'MO', 'DFS', 'ADP', 'TAP', 'AIG', 'GRMN', 'KO', 'TROW', 'CL', 'KMB', 'CLX', 'UDR', 'ED', 'ZTS', 'PCG', 'ACN', 'EBAY', 'OKE', 'CME', 'VNO', 'NEM', 'WHR', 'PFE', 'HSY', 'UAA', 'MET', 'FDX', 'PEG', 'MON', 'XEL', 'PPG', 'SLB', 'UPS', 'TPR', 'GPS', 'SYF', 'FCX', 'TSN', 'ABT', 'DOV', 'MYL', 'RL', 'LYB', 'BK', 'PXD', 'GLW', 'ANTM', 'SYY', 'GPC', 'MDLZ', 'CINF', 'GD', 'EXPD', 'NOV', 'BAX', 'AEE', 'L', 'ULTA', 'COTY', 'ECL', 'USB', 'VIAB', 'UNM', 'PX', 'MRO', 'SPGI', 'MAT', 'PRGO', 'WMB', 'PHM', 'GT', 'HIG', 'DRE', 'CTL', 'ADM', 'TMK', 'KMI', 'CMS', 'AIV', 'HPE', 'HPQ', 'BSX', 'FLR', 'TWX', 'AET', 'SYMC', 'HRL', 'STZ', 'JNPR', 'WY', 'CHK', 'RRC', 'KEY', 'COG', 'NRG', 'NWS', 'FOXA', 'F', 'SWK', 'HUM', 'AVGO', 'DGX', 'MOS'), value='AMZN'), Output()), _dom_classes=('widget-interact',))

In [23]:
def getCalls2(optionPrices):
    callPrices = []
    for i in optionPrices:
        if i['Call'] != None:
            callPrices.append(i[0])
    return callPrices

In [24]:
def getPricesByRiskFreeRate(optionPrices, RiskFreeRate):
    allPrices = []
    for i in optionPrices[0][0]:
        if i['RiskFreeRate'] == RiskFreeRate:
            allPrices.append(i)
    return allPrices

In [25]:
def graphCallByTickerWithRate(tickerSymbol, rate):
    prices = getOptionPricesByTicker(tickerSymbol)
    ratePrices = getPricesByRiskFreeRate(prices,rate)
    prices = getCalls2(ratePrices)
    opPriceDic = priceToArray2(prices)
    numOfDays = sparkSession.sql("SELECT NumberOfDays FROM optionData WHERE Ticker Like '" + tickerSymbol + "'").collect()
    numberOfDays = getNumberDaysArray(numOfDays)
    for key, values in opPriceDic.items(): 
        plt.plot(numberOfDays, values)
    plt.xlabel('Days to Expiration')
    plt.ylabel('Option Prices')
    plt.title('Call Options for '+ tickerSymbol + ' with risk free rate ' + str(rate))
    plt.ticklabel_format(style='plain')
    plt.show()
    
    
# numOfDays = sparkSession.sql("SELECT NumberOfDays FROM optionData WHERE Ticker Like '" + tickerSymbol + "'").collect()
# numberOfDays = getNumberDaysArray(numOfDays)
# plt.plot(numberOfDays, opPriceDic)
# plt.xlabel('Days to Expiration')
# plt.ylabel('Option Prices')
# plt.title('Call Options for '+ tickerSymbol + ' with risk free rate ' + str(rate))
# plt.show()

In [26]:
def passRateCall(Ticker):
    riskFreeRates = getRiskRateForGraph(Ticker)
    interact(graphCallByTickerWithRate,  tickerSymbol=fixed(Ticker), rate = riskFreeRates);

In [ ]:
interact(passRateCall, Ticker=tickerSymbols);

interactive(children=(Dropdown(description='Ticker', options=('AMZN', 'BRK.B', 'GOOG', 'BA', 'MLM', 'XL', 'HSIC', 'MTD', 'CMG', 'IDXX', 'PDCO', 'FLIR', 'CMA', 'INTU', 'KHC', 'FL', 'ZION', 'ROK', 'MAS', 'EL', 'ARNC', 'FOX', 'ADBE', 'HLT', 'RE', 'MCHP', 'JCI', 'BHGE', 'LLL', 'SPG', 'DVA', 'TJX', 'ALK', 'XRX', 'ALL', 'TRV', 'PPL', 'DRI', 'MSI', 'SEE', 'FMC', 'ANSS', 'HRB', 'LNC', 'INCY', 'ABBV', 'MTB', 'EQIX', 'WM', 'AVB', 'RTN', 'EIX', 'ALB', 'GGP', 'HBI', 'FTI', 'IVZ', 'DUK', 'UHS', 'PCLN', 'C', 'NTRS', 'CCI', 'AON', 'COL', 'FE', 'EXPE', 'PH', 'QRVO', 'AYI', 'CPB', 'SO', 'AVY', 'SJM', 'DG', 'NWL', 'D', 'LEN', 'SIG', 'DLR', 'GIS', 'WLTW', 'ESS', 'OMC', 'TEL', 'VRSN', 'AAP', 'JWN', 'CELG', 'BWA', 'ARE', 'IPG', 'FISV', 'ORLY', 'MU', 'JEC', 'EQR', 'CNC', 'CTAS', 'SBAC', 'APH', 'NFLX', 'LB', 'STX', 'RMD', 'WRK', 'BXP', 'PSA', 'SYK', 'SRE', 'PVH', 'PKG', 'KLAC', 'TMO', 'EMN', 'VMC', 'GOOGL', 'NOC', 'HBAN', 'STI', 'AMP', 'SNA', 'PNC', 'CB', 'AZO', 'TDG', 'ROST', 'CHRW', 'DHR', 'XYL', 'MCO', 'IR', 'WDC', 'BCR', 'CXO', 'NFX', 'K', 'FITB', 'PRU', 'A', 'BEN', 'NVDA', 'DWDP', 'APD', 'WYN', 'AGN', 'NI', 'HRS', 'AOS', 'IT', 'ADS', 'SWKS', 'CTXS', 'NDAQ', 'HP', 'LH', 'HST', 'PAYX', 'EXC', 'ROP', 'UTX', 'VTR', 'IRM', 'PWR', 'CA', 'RSG', 'WMT', 'COO', 'JBHT', 'O', 'ADSK', 'XEC', 'ICE', 'RF', 'NEE', 'FB', 'PFG', 'XOM', 'DISCA', 'STT', 'CDNS', 'CBG', 'CFG', 'HCP', 'MSFT', 'ZBH', 'MHK', 'ETR', 'EQT', 'LOW', 'CSX', 'FIS', 'RJF', 'AMG', 'XRAY', 'LRCX', 'IFF', 'HOLX', 'PCAR', 'GPN', 'CVS', 'EMR', 'SCG', 'FLS', 'CERN', 'CRM', 'CAG', 'PYPL', 'ISRG', 'FAST', 'NCLH', 'DTE', 'KIM', 'DXC', 'MKC', 'NLSN', 'AAPL', 'REG', 'BIIB', 'MMC', 'INTC', 'PNW', 'QCOM', 'PGR', 'HCN', 'AMAT', 'CI', 'RHT', 'PSX', 'CHD', 'ATVI', 'AVGO', 'REGN', 'GS', 'NKE', 'NWSA', 'WEC', 'WFC', 'UAL', 'TGT', 'PNR', 'DLTR', 'NBL', 'BLK', 'BBT', 'KSU', 'UNP', 'ALLE', 'WYNN', 'CAT', 'AEP', 'JPM', 'SNPS', 'JNJ', 'AWK', 'ALGN', 'EVHC', 'ETN', 'WAT', 'NTAP', 'HAS', 'SRCL', 'LMT', 'PG', 'FRT', 'APA', 'ORCL', 'DIS', 'DGX', 'EOG', 'AFL', 'EA', 'NUE', 'AAL', 'WBA', 'CTSH', 'VLO', 'HD', 'SBUX', 'CVX', 'COF', 'TSS', 'OXY', 'V', 'AXP', 'BDX', 'DISCK', 'MMM', 'NAVI', 'HAL', 'LUV', 'BHF', 'AKAM', 'ANDV', 'ABC', 'BBY', 'AJG', 'INFO', 'CSCO', 'TXN', 'LNT', 'AIZ', 'MCD', 'APC', 'MS', 'M', 'MDT', 'ES', 'ESRX', 'DE', 'BAC', 'ILMN', 'DLPH', 'PEP', 'SHW', 'ADI', 'GM', 'RCL', 'KSS', 'CSRA', 'DISH', 'DHI', 'ITW', 'IBM', 'DPS', 'GWW', 'MNST', 'SCHW', 'MAR', 'KORS', 'EXR', 'KR', 'TIF', 'BLL', 'CF', 'MA', 'VAR', 'PM', 'MAA', 'MAC', 'AME', 'PKI', 'DAL', 'AMGN', 'LEG', 'CHTR', 'UNH', 'HES', 'URI', 'TXT', 'CCL', 'COP', 'MPC', 'ETFC', 'MRK', 'VRTX', 'COST', 'TSCO', 'FFIV', 'CBOE', 'PBCT', 'WU', 'BMY', 'AMT', 'GILD', 'NSC', 'LKQ', 'AMD', 'LLY', 'CAH', 'ALXN', 'YUM', 'CBS', 'HOG', 'VZ', 'CMCSA', 'SLG', 'MCK', 'VRSK', 'VFC', 'LUK', 'PLD', 'FBHS', 'RHI', 'SNI', 'AES', 'CMI', 'GE', 'CNP', 'EFX', 'UA', 'T', 'XLNX', 'TRIP', 'FTV', 'HON', 'MGM', 'KMX', 'IP', 'HCA', 'EW', 'DVN', 'MO', 'DFS', 'ADP', 'TAP', 'AIG', 'GRMN', 'KO', 'TROW', 'CL', 'KMB', 'CLX', 'UDR', 'ED', 'ZTS', 'PCG', 'ACN', 'EBAY', 'OKE', 'CME', 'VNO', 'NEM', 'WHR', 'PFE', 'HSY', 'UAA', 'MET', 'FDX', 'PEG', 'MON', 'XEL', 'PPG', 'SLB', 'UPS', 'TPR', 'GPS', 'SYF', 'FCX', 'TSN', 'ABT', 'DOV', 'MYL', 'RL', 'LYB', 'BK', 'PXD', 'GLW', 'ANTM', 'SYY', 'GPC', 'MDLZ', 'CINF', 'GD', 'EXPD', 'NOV', 'BAX', 'AEE', 'L', 'ULTA', 'COTY', 'ECL', 'USB', 'VIAB', 'UNM', 'PX', 'MRO', 'SPGI', 'MAT', 'PRGO', 'WMB', 'PHM', 'GT', 'HIG', 'DRE', 'CTL', 'ADM', 'TMK', 'KMI', 'CMS', 'AIV', 'HPE', 'HPQ', 'BSX', 'FLR', 'TWX', 'AET', 'SYMC', 'HRL', 'STZ', 'JNPR', 'WY', 'CHK', 'RRC', 'KEY', 'COG', 'NRG', 'NWS', 'FOXA', 'F', 'SWK', 'HUM', 'AVGO', 'DGX', 'MOS'), value='AMZN'), Output()), _dom_classes=('widget-interact',))

In [ ]:
# def graph(ticker):    
#     # Code for put by ticker
#     data = []
#     opPriceDic = priceToArray(getPuts(getOptionPricesByTicker(ticker)))
#     for key, values in opPriceDic.items():
#         trace = go.Scatter(
#             x = numberOfDays,
#             y = values,
#             name = 'option',
#             line = dict(
#                 width = 4)
#         )
#         data.append(trace)
#     # Edit the layout
#     layout = dict(title = 'Call prices for ' ,
#                   xaxis = dict(title = 'Days to Expiration'),
#                   yaxis = dict(title = 'Price '),
#                   )
#     fig = dict(data=data, layout=layout)
#     py.iplot(fig, filename='calls For ')

In [ ]:
# Code for calls by ticker
# data = []
# opPriceDic = priceToArray(getCalls(getOptionPricesByTicker('ABBV')))
# for key, values in opPriceDic.items():
#     trace = go.Scatter(
#         x = numberOfDays,
#         y = values,
#         name = 'option',
#         line = dict(
#             width = 4)
#     )
#     data.append(trace)
# # Edit the layout
# layout = dict(title = 'Call prices for ' ,
#               xaxis = dict(title = 'Days to Expiration'),
#               yaxis = dict(title = 'Price '),
#               )
# fig = dict(data=data, layout=layout)
# py.iplot(fig, filename='calls For ')

In [ ]:

# p.tools.set_credentials_file(username='lalor188', api_key='MQxJd3Rs9X5aKOZiDEgt') # Plotly # Not gonna be using to remove
# optionTicker = widgets.Dropdown( # not gonna work remove
# #     options=tickerSymbols,
# #     value=tickerSymbols[0],
# #     description='Option Ticker symbol:',
# # )
# # g = GraphWidget('https://plot.ly/~kevintest/1192/')

        
# # # def response2(change): # not gonna work remove
# # #     # print "using date"
# # #     if optionTicker.value:
# # #         filter_list = optionTicker.value
# # #         data = []
# # #         yData = []
# # #         opPriceDic = priceToArray(getPuts(getOptionPricesByTicker(filter_list)))
# # #         x = numberOfDays
# # #         for key, values in opPriceDic.items():
# # #             yData.append(values.copy())
# # #         g.restyle({'x': [numberOfDays],'y': [yData],  'type': 'line' })
# # #         g.relayout({'barmode': 'overlay', 'xaxis': {'title': 'Days to expiration'},
# # #                 'yaxis': {'title': 'Price ($)'}})
    
# # # optionTicker.observe(response2, names="value") # not gonna work remove all
# # # # container2 = widgets.HBox([optionTicker]) 
# # # # display(container2)
# # # # display(g)

In [105]:
# demo purposes To remove
# import plotly.plotly as py 
# import plotly.graph_objs as go 



# data = []
# opPriceDic = priceToArray(getPuts(getOptionPricesByTicker('ZTS')))
# for key, values in opPriceDic.items():
#     trace = go.Scatter(
#         x = numberOfDays,
#         y = values,
#         name = 'option',
#         line = dict(
#             width = 4)
#     )
#     data.append(trace)

# high_annotations=[dict(x='2016-03-01',
#                        y=5,
#                        xref='x', yref='y',
#                        text='High Average:<br>'+str(6),
#                        ax=0, ay=-40),
#                   dict(x=5,
#                        y=5,
#                        xref='x', yref='y',
#                        text='High Max:<br>'+str(6),
#                        ax=0, ay=-40)]
# low_annotations=[dict(x='2015-05-01',
#                       y=1,
#                       xref='x', yref='y',
#                       text='Low Average:<br>'+str(1),
#                       ax=0, ay=40),
#                  dict(x=1,
#                       y=1,
#                       xref='x', yref='y',
#                       text='Low Min:<br>'+str(1),
#                       ax=0, ay=40)]

# updatemenus = list([
#     dict(active=-1,
#          buttons=list([   
#             dict(label = 'MMM',
#                  method = 'update',
#                  args = [{'visible': [True, True, False, False]},
#                          {'title': 'Yahoo High',
#                           'annotations': high_annotations}]),
#             dict(label = 'Low',
#                  method = 'update',
#                  args = [{'visible': [False, False, True, True]},
#                          {'title': 'Yahoo Low',
#                           'annotations': low_annotations}]),
#             dict(label = 'Both',
#                  method = 'update',
#                  args = [{'visible': [True, True, True, True]},
#                          {'title': 'Yahoo',
#                           'annotations': high_annotations+low_annotations}]),
#             dict(label = 'Reset',
#                  method = 'update',
#                  args = [{'visible': [True, False, True, False]},
#                          {'title': 'Yahoo',
#                           'annotations': []}])
#         ]),
#     )
# ])

# layout = dict(title='Yahoo', showlegend=False,
#               updatemenus=updatemenus)

# fig = dict(data=data, layout=layout)
# py.iplot(fig, filename='update_dropdown')